# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: `«Смарт»` или `«Ультра»`.

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы.

**Цель исследования:** Построить модель для задачи классификации, которая выберет подходящий тариф. При этом, доля правильных ответов должна быть не менее 0.75.

**Ход исследования:** 
- Откроем предложенный датасет и познакомимся с общей информацией;
- Разделим исходные данные на обучающую, валидационную и тестовую выборки;
- Исследуем качество разных моделей, меняя гиперпараметры;
- Проверьте качество модели на тестовой выборке.

## Изучение файла

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score 

Прочитайте файлs формата .csv из папки /datasets и сохраним их в переменной df

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

Выведем на экран первые 5 строк таблицs, а также посмотрим общую информацию

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


В таблице присутствуют пять столбцов. Четыре стобца имеют тип данных `float`, один тип данных `int`.

Согласно документации к данным:

- `сalls` — количество звонков,
- `minutes` — суммарная длительность звонков в минутах,
- `messages` — количество sms-сообщений,
- `mb_used` — израсходованный интернет-трафик в Мб,
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Разбитие данных на выборки

Разобьем данные на три части: обучающую, валидационную и тестовую. Обучающая выборка нужна нам для обучения модели. Валидация подсказывает, как ведёт себя модель в полевых условиях и нет ли переобучения, возможно какие-то параметры стоит изменить. Тестовая выборка нужна нам, чтобы правильно оценить готовую модель.

Размеры валидационного и тестового наборов определим как равные. Исходные данные разобьем в соотношении 3:1:1.

In [5]:
df.shape # изучим размер исходной таблицы

(3214, 5)

In [6]:
df_train, df_valid_test = train_test_split(df, test_size=0.4, random_state=12345) # обучающая=0.6, валид + тест = 0.4
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=12345) # валид и тест модели делим пополам

Проверим размеры выборок, чтобы проверить в правильном ли соотношении они распределены.

In [7]:
df_train.shape

(1928, 5)

In [8]:
df_valid.shape

(643, 5)

In [9]:
df_test.shape

(643, 5)

## Исследование моделей

### Решающее дерево

Исследуем первую модель: `решающее дерево`. В ней выстраивается дерево с ответами «Да»/«Нет» и различными вариантами действий. 

Обучающий и валидационный набор данных сохраним в переменных.

In [10]:
features_train = df_train.drop(['is_ultra'], axis=1) # признаки
target_train = df_train['is_ultra'] # целевой признак

In [11]:
features_valid = df_valid.drop(['is_ultra'], axis=1) 
target_valid = df_valid['is_ultra']  

Создаём объект этой структуры данных. В переменной model будет храниться модель.

In [12]:
model = DecisionTreeClassifier() 

In [13]:
model.fit(features_train, target_train) # выполняем обучение 
train_predictions = model.predict(features_train) # предсказание

Возьмём валидационную выборку. Проверим, как модель справится.

In [14]:
valid_predictions = model.predict(features_valid)

Основной метрикой качества модели является отношение числа правильных ответов к размеру тестируемой выборки и называется `accuracy`.

In [15]:
accuracy_score(target_train, train_predictions) 

1.0

In [16]:
accuracy_score(target_valid, valid_predictions) 

0.7216174183514774

Модель идеально объясняла примеры из обучающего набора данных, но в валидационной выборке результат снизился. Это свидетельствует о проблеме переобучения. Попробуем изменить глубину дерева (максимальное количество условий от «вершины» до финального ответа).

In [17]:
model = DecisionTreeClassifier(random_state=12345, max_depth=3)
model.fit(features_train, target_train) 
train_predictions = model.predict(features_train)

In [18]:
valid_predictions = model.predict(features_valid)

In [19]:
accuracy_score(target_train, train_predictions) 

0.8075726141078838

In [20]:
accuracy_score(target_valid, valid_predictions) 

0.7853810264385692

Здесь модель показывает более высокое качество. Проверим несколько вариантов глубины дерева, чтобы понять какая подходит нам лучше.

In [21]:
best_model = None
best_result = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучим модель с заданным количеством деревьев
    model.fit(features_train, target_train) # обучим модель на тренировочной выборке
    result = model.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
    if result > best_result:
        best_depth = depth # сохраним лучшее значение глубины
        best_model = model # сохраним наилучшую модель
        best_result = result #  сохраним наилучшее значение метрики accuracy на валидационных данных

In [22]:
best_depth

3

In [23]:
best_result

0.7853810264385692

### Случайный лес

Исследуем модель `случайный лес`. Алгоритм обучает большое количество независимых друг от друга деревьев, а потом принимает решение на основе голосования. Случайный лес помогает улучшить результат предсказания и избежать переобучения.

Обучим модель и проверим результаты на валидационной выборке.

In [24]:
model = RandomForestClassifier(random_state=12345, n_estimators=3) 

In [25]:
model.fit(features_train, target_train) 

RandomForestClassifier(n_estimators=3, random_state=12345)

In [26]:
model.predict(features_train) 

array([0, 0, 0, ..., 1, 1, 1])

Правильность модели проверим методом score(). Он считает accuracy для всех алгоритмов классификации.

In [27]:
model.score(features_train, target_train) 

0.9507261410788381

In [28]:
model.predict(features_valid) 
model.score(features_valid, target_valid) 

0.7387247278382582

Подберем оптимальное количество деревьев.

In [45]:
best_model = None
best_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # обучим модель с заданным количеством деревьев
    model.fit(features_train, target_train) # обучим модель на тренировочной выборке
    result = model.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
    if result > best_result:
        best_est = est  # сохраним лучшее значение количества деревьев
        best_model = model # сохраним наилучшую модель
        best_result = result #  сохраним наилучшее значение метрики accuracy на валидационных данных

In [46]:
best_est

10

In [47]:
best_result

0.7853810264385692

### Логистическая регрессия

Обучим модель. Добавим дополнительные гиперпараметры: solver='lbfgs' и max_iter=1000. Первый гиперпараметр позволяет выбрать алгоритм, который будет строить модель. Алгоритм `'lbfgs'` — один из самых распространённых. Гиперпараметром `max_iter` задаётся максимальное количество итераций обучения.

In [32]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 

In [33]:
model.fit(features_train, target_train) 

LogisticRegression(max_iter=1000, random_state=12345)

In [34]:
model.predict(features_train) 

array([0, 0, 0, ..., 0, 0, 0])

In [35]:
model.score(features_train, target_train) 

0.7131742738589212

In [36]:
model.predict(features_valid) 
model.score(features_valid, target_valid) 

0.7107309486780715

Рассмотрим влияет ли величина max_iter на рассчеты

In [48]:
best_model = None
best_result = 0
for optimal_iter in range(100, 2000, 200):
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=optimal_iter) 
    model.fit(features_train, target_train) # обучим модель на тренировочной выборке
    result = model.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
    if result > best_result:
        best_iter = optimal_iter  # сохраним лучшее значение количества итераций обучения
        best_model = model # сохраним наилучшую модель
        best_result = result #  сохраним наилучшее значение метрики accuracy на валидационных данных

In [49]:
best_iter

100

In [50]:
best_result

0.7107309486780715

## Проверка модели на тестовой выборке

Сохраним тестовый набор данных в переменных.

In [40]:
features_test = df_test.drop(['is_ultra'], axis=1) 
target_test = df_test['is_ultra']  

Проверим результаты работы моделей на тестовой выборке данных

Модель случайного леса

In [42]:
model = RandomForestClassifier(random_state=12345, n_estimators=10) 
model.fit(features_train, target_train) 
test_predictions = model.predict(features_test) 
model.score(features_test, target_test) 

0.7807153965785381

Две из трех моделе показали необходимый нам уровень качества. Существует еще один важный показатель: скорость работы. Значения accuracy у `решающего дерева` и `случайного леса` достаточно близки, но при этом скорость работы решающего дерева значительно выше. Но так, как в нашем случае скорость работы моделей не сильно заметно (оба расчета производятся достаточно быстро), выберем модель с наивысшим показателем качества: **модель случайного леса**. 

## Вывод

Мы исследовали данные о поведении клиентов, которые уже перешли на тарифы `«Смарт»` и `«Ультра»`. Нашей целью было выбрать лучшую модель для подбора оптимального тарифа клиенту.
 
В начале исследования мы открыли предложенный датасет и познакомимся с общей информацией. 

Далее разделили исходные данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1. Обучающая выборка нужна нам для обучения модели. Валидация подсказывает, как ведёт себя модель в полевых условиях. Тестовая выборка нужна нам, чтобы правильно оценить готовую модель.

Затем мы исследовали качество нескольких моделей: модель решающего дерева, модель случайного леча, модель логистической регрессии. Подобрали оптимальные гиперпараметры для работы с нашим датасетом

В завершении исследования проверили работу моделей на тестовой выборке, отсекли модель, которая несоответствовала необходимому условию (accuracy > 0.75), из оставшихся выбрали модель с наивысшим уровнем качества предсказания: Модель случайного леса. В будущем при увеличении объема обучающих данных, возможно изменить свой выбор в пользу модели решающего леса, в связи с более высокой скоростью выполнения.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
